In [1]:
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('./result.csv')
print(df.shape)

# LIMIT = 10
# df = df[:LIMIT]
print(df.shape)
df

(26050, 3)
(26050, 3)


,category,target_name,content
0,malicious,legitimate-injected/3/notes.md,"ParseResult(scheme='', netloc='', path='<scrip..."
1,malicious,neutrino/201603_neutrino_lp.html,"ParseResult(scheme='', netloc='', path='<scrip..."
2,malicious,neutrino/2016-02-12-Neutrino-EK-landing-page_i...,"ParseResult(scheme='', netloc='', path='<scrip..."
3,malicious,neutrino/201601_neutrino_lp.html,"ParseResult(scheme='', netloc='', path='<scrip..."
4,malicious,neutrino/201602_neutrino_lp.html,"ParseResult(scheme='', netloc='', path='<scrip..."
...,...,...,...
26045,crypto,abc8/linkedin,"ParseResult(scheme='', netloc='', path='<scrip..."
26046,crypto,abc8/linkedin,"ParseResult(scheme='', netloc='', path='<scrip..."
26047,crypto,abc8/linkedin,"ParseResult(scheme='', netloc='', path='<scrip..."
26048,crypto,abc8/linkedin,"ParseResult(scheme='', netloc='', path='<scrip..."


In [3]:
from textblob import Word
from nltk.corpus import stopwords
import nltk

In [4]:
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
def clean_text(text):
    return " ".join([ Word(word).lemmatize() for word in re.sub("[^A-Za-z0-9]+", " ", text).lower().split() if word not in stopword])

In [ ]:
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
import seaborn as sns
perp_components = defaultdict(dict)
for i in [10,30,50,100]:
    for ngram in [(1,1),(1,2),(1,3),(2,2),(3,3)]:
        vectorizer = TfidfVectorizer(ngram_range=ngram, max_features= 2**10)
        text_to_vector = vectorizer.fit_transform(df.content.values)
        print("Ngram ",ngram )
        print("Perplexity ", i)
        X_embedded = TSNE(perplexity=i ).fit_transform(text_to_vector)
        ngram_str = str(ngram[0])+"_"+str(ngram[1])
        perp_components[i][ngram_str] = X_embedded
        # sns settings
        sns.set(rc={'figure.figsize':(15,15)})
        # colors
        palette = sns.color_palette("hls", len(set(df.target_name.values.tolist())))        
        y = df.target_name.values.tolist()
        # plot
#         sns.scatterplot(X_embedded[:,0], X_embedded[:,1], hue=y, palette=palette)
        sns.scatterplot(X_embedded[:,0], X_embedded[:,1], hue=y, legend=False, palette=palette)
        title = "t-SNE- JS group - TfIdf - "+ngram_str+"- tSNE perplexity - "+str(i)
#         plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1));
        plt.savefig(title)
        plt.title(title)
        plt.show()

Ngram  (1, 1)
Perplexity  10
